# 이미지-캡션 쌍 데이터셋 만들기 및 사용하기 튜토리얼

출처 : https://huggingface.co/docs/diffusers/training/create_dataset

(24.07.23, 작성자 유소영)

**1. 소개** : 이미지-캡션 쌍 데이터셋은 이미지와 그에 해당하는 설명(캡션)을 쌍으로 묶은 데이터셋임. \
**2. 데이터셋 구조 설명** : 이미지-캡션 쌍 데이터셋의 일반적인 구조는 다음과 같다:
    
```
dataset_folder/
    ├── train/
    │   ├── metadata.jsonl
    │   ├── image1.jpg
    │   ├── image2.jpg
    │   └── ...
    └── validation/
        ├── metadata.jsonl
        ├── image1.jpg
        ├── image2.jpg
        └── ...

```

<u> 여기서 metadata.jsonl 파일은 각 이미지에 대한 캡션 정보를 포함하고 있음. </u>

```
{"file_name": "image1.jpg", "text": "A cat sitting on a windowsill"}
{"file_name": "image2.jpg", "text": "A dog running in the park"}

```

    각 줄은 JSON 객체로, file_name은 이미지 파일의 이름을, text는 해당 이미지의 캡션을 나타냄.


**3. metadata.jsonl** 파일 생성 방법: 
- **수동 생성 방법** : 작은 데이터셋의 경우, 텍스트 편집기를 사용하여 직접 metadata.jsonl 파일을 만들 수 있습니다. 각 줄에 JSON 형식으로 파일 이름과 캡션을 입력하면 됩니다.
- **자동화 스크립트 예시** : 대규모 데이터셋의 경우, Python 스크립트를 사용하여 metadata.jsonl 파일을 자동으로 생성할 수 있습니다. ***다음은 간단한 예시 스크립트입니다***:

In [ ]:
import os
import json
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image

def create_metadata_jsonl(folder_path):
    image_extensions = ['.png', '.jpg', '.jpeg', '.gif', '.bmp']
    metadata = []

    # 폴더 내의 모든 이미지 파일 찾기
    image_files = [f for f in os.listdir(folder_path) if any(f.lower().endswith(ext) for ext in image_extensions)]

    for i, file in enumerate(image_files):
        file_path = os.path.join(folder_path, file)
        
        # 이미지 표시
        #img = Image.open(file_path)
        #img = img.resize((200,200))
        #img.show()
        
        # 사용자로부터 이미지 캡션 입력 받기
        caption = input(f"Enter caption for {file} (or 'q' to quit): ")

        if caption.lower() == 'q':
            break
        
        # 메타데이터 딕셔너리 생성
        metadata_item = {
            "file_name": file,
            "text": caption
        }
        metadata.append(metadata_item)

    # metadata.jsonl 파일 생성
    output_path = os.path.join(folder_path, "metadata.jsonl")
    with open(output_path, 'w', encoding='utf-8') as f:
        for item in metadata:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')

    print(f"metadata.jsonl has been created at {output_path}")
    
folder_path = './data/train/Wheel/'
create_metadata_jsonl(folder_path)

**4. Hugging Face datasets 라이브러리를 사용한 데이터셋 로드**

   Hugging Face의 datasets 라이브러리는 ImageFolder 기능을 통해 이미지-캡션 쌍 데이터셋을 쉽게 로드할 수 있게 해줍니다.
   
   ImageFolder 사용 방법
   먼저, datasets 라이브러리를 설치합니다:

```
pip install datasets
```

   그런 다음, 아래 스크립트를 통해 ***로컬에 저장된 데이터셋***을 로드할 수 있습니다:

In [ ]:
from datasets import load_dataset

# 데이터셋 로드
dataset = load_dataset("./data/")

# 이미지와 캡션 출력
import matplotlib.pyplot as plt

In [ ]:
for i in range(10):
    sample = dataset['train'][i]
    plt.imshow(sample['image'])
    plt.axis('off')
    plt.title(sample['text'])
    plt.show()